## Getting resources

We use [xp5k](https://github.com/pmorillon/xp5k) to manage the interaction with the Grid resources.

We create a new ```XP``` and define a new job.

Note that ```resources``` are the same as those given in a ```oarsub``` command line.

In [1]:
require 'xp5k'
myxp = XP5K::XP.new
myxp.define_job({
    :resources => ["nodes=50, walltime=1:00:00"],
    :site      => 'lyon',
    :types     => ["allow_classic_ssh"],
    :name      => "iruby notebook" ,
    :roles     => [XP5K::Role.new({:name => 'mynode', :size => 50})],
    :command   => "sleep 86400"
    })

## Submission

```submit``` is asynchronous, ```wait_for_jobs``` will wait for all the resources to be ready.

In [2]:
myxp.submit
myxp.wait_for_jobs

#<Proc:0x007fabaf947098@/Users/msimonin/msimonin@github.com/xp5k/lib/xp5k/xp.rb:118>

## Run command

```xp5k/rake``` include some helpers to run commands on the nodes of your reservation.

In [3]:
require 'xp5k/rake'
h = on roles('mynode'), {:user => 'msimonin', :capture => true} do 
    cmd = ['uptime -s']
end

Connected to sagittaire-18.lyon.grid5000.fr...
Connected to sagittaire-20.lyon.grid5000.fr...
Connected to sagittaire-2.lyon.grid5000.fr...
Connected to sagittaire-22.lyon.grid5000.fr...
Connected to sagittaire-28.lyon.grid5000.fr...
Connected to sagittaire-3.lyon.grid5000.fr...
Connected to sagittaire-29.lyon.grid5000.fr...
Connected to sagittaire-30.lyon.grid5000.fr...
Connected to sagittaire-24.lyon.grid5000.fr...
Connected to sagittaire-19.lyon.grid5000.fr...
Connected to sagittaire-33.lyon.grid5000.fr...
Connected to sagittaire-31.lyon.grid5000.fr...
Connected to sagittaire-32.lyon.grid5000.fr...
Connected to sagittaire-34.lyon.grid5000.fr...
Connected to sagittaire-46.lyon.grid5000.fr...
Connected to sagittaire-47.lyon.grid5000.fr...
Connected to sagittaire-5.lyon.grid5000.fr...
Connected to sagittaire-50.lyon.grid5000.fr...
Connected to sagittaire-53.lyon.grid5000.fr...
Connected to sagittaire-55.lyon.grid5000.fr...
Connected to sagittaire-56.lyon.grid5000.fr...
Connected to sag

{"sagittaire-3.lyon.grid5000.fr"=>["2016-01-13 23:33:27"], "sagittaire-2.lyon.grid5000.fr"=>["2016-01-13 17:47:00"], "sagittaire-18.lyon.grid5000.fr"=>["2016-01-13 17:46:48"], "sagittaire-22.lyon.grid5000.fr"=>["2016-01-13 23:33:40"], "sagittaire-28.lyon.grid5000.fr"=>["2016-01-13 23:33:31"], "sagittaire-30.lyon.grid5000.fr"=>["2016-01-13 23:33:31"], "sagittaire-24.lyon.grid5000.fr"=>["2016-01-13 23:33:28"], "sagittaire-20.lyon.grid5000.fr"=>["2016-01-13 23:33:28"], "sagittaire-19.lyon.grid5000.fr"=>["2016-01-13 23:33:28"], "sagittaire-29.lyon.grid5000.fr"=>["2016-01-13 23:33:32"], "sagittaire-31.lyon.grid5000.fr"=>["2016-01-13 23:33:48"], "sagittaire-34.lyon.grid5000.fr"=>["2016-01-13 23:33:28"], "sagittaire-32.lyon.grid5000.fr"=>["2016-01-13 23:33:31"], "sagittaire-33.lyon.grid5000.fr"=>["2016-01-13 23:33:47"], "sagittaire-38.lyon.grid5000.fr"=>["2016-01-13 23:33:29"], "sagittaire-39.lyon.grid5000.fr"=>["2016-01-13 23:33:30"], "sagittaire-4.lyon.grid5000.fr"=>["2016-01-13 23:33:27"],

## Draw the bar plot

The output of the above command is a map :

 * each key is a server name 
 * a value is the outputs of the commands run on this server
 
e.g : ``` 'sagittaire-1.lyon.grid5000.fr' => ['out1', ... , 'outn'] ```

The following code compute the time since last reboot in minute.

In [4]:
require 'nyaplot'
require 'time'

now = Time.now.to_i
plot = Nyaplot::Plot.new
df = Nyaplot::DataFrame.new({
    host: h.keys
           .map{|x| x.split('.').first},
    uptime: h.values
             .map{|x| x.first}
    .map{|x| ((now - Time.parse(x).to_f)/60).round(2)}
    })
    

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    window.dispatchEvent(event);\n\t    console.log('Finished loading Nyaplotjs');\n\n\t};\n\n\n});});\n}\n"

host,uptime
sagittaire-3,39.43
sagittaire-2,385.88
sagittaire-18,386.08
sagittaire-22,39.22
sagittaire-28,39.37
sagittaire-30,39.37
sagittaire-24,39.42
sagittaire-20,39.42
sagittaire-19,39.42
sagittaire-29,39.35


In [5]:
plot=Nyaplot::Plot.new
plot.add_with_df(df, :bar, :host, :uptime)
plot.configure do
    x_label('')
    y_label('Uptime (min)')
    rotate_x_label(-90)
end


#<Nyaplot::Plot:0x007fabab7e7148 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007fabab7e6fb8 @properties={:type=>:bar, :options=>{:x=>:host, :y=>:uptime}, :data=>"6f217fab-34eb-43b0-b2f2-e3cbc884a282"}, @xrange=["sagittaire-3", "sagittaire-2", "sagittaire-18", "sagittaire-22", "sagittaire-28", "sagittaire-30", "sagittaire-24", "sagittaire-20", "sagittaire-19", "sagittaire-29", "sagittaire-31", "sagittaire-34", "sagittaire-32", "sagittaire-33", "sagittaire-38", "sagittaire-39", "sagittaire-4", "sagittaire-42", "sagittaire-45", "sagittaire-43", "sagittaire-46", "sagittaire-47", "sagittaire-48", "sagittaire-49", "sagittaire-5", "sagittaire-52", "sagittaire-50", "sagittaire-53", "sagittaire-54", "sagittaire-55", "sagittaire-56", "sagittaire-57", "sagittaire-58", "sagittaire-6", "sagittaire-63", "sagittaire-64", "sagittaire-65", "sagittaire-66", "sagittaire-61", "sagittaire-59", "sagittaire-67", "sagittaire-69", "sagittaire-7", "sagittaire-71", "sagittaire-72", "sagittaire-74", "sagittaire-75", "sagittaire-77", "sagittaire-78", "sagittaire-9"], @yrange=[0, 386.08]>], :options=>{:x_label=>"", :y_label=>"Uptime (min)", :rotate_x_label=>-90, :width=>700, :xrange=>["sagittaire-3", "sagittaire-2", "sagittaire-18", "sagittaire-22", "sagittaire-28", "sagittaire-30", "sagittaire-24", "sagittaire-20", "sagittaire-19", "sagittaire-29", "sagittaire-31", "sagittaire-34", "sagittaire-32", "sagittaire-33", "sagittaire-38", "sagittaire-39", "sagittaire-4", "sagittaire-42", "sagittaire-45", "sagittaire-43", "sagittaire-46", "sagittaire-47", "sagittaire-48", "sagittaire-49", "sagittaire-5", "sagittaire-52", "sagittaire-50", "sagittaire-53", "sagittaire-54", "sagittaire-55", "sagittaire-56", "sagittaire-57", "sagittaire-58", "sagittaire-6", "sagittaire-63", "sagittaire-64", "sagittaire-65", "sagittaire-66", "sagittaire-61", "sagittaire-59", "sagittaire-67", "sagittaire-69", "sagittaire-7", "sagittaire-71", "sagittaire-72", "sagittaire-74", "sagittaire-75", "sagittaire-77", "sagittaire-78", "sagittaire-9"], :yrange=>[0, 386.08]}}>

In [ ]:
myxp.clean